In [1]:
from datetime import datetime, timedelta
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import os

from collections import Counter

## Analyse data

In [2]:
df_sum = None

file_path_list =list(Path("../data/local/tables/").iterdir())[:]
for df_path in tqdm(list(Path("../data/local/tables/").iterdir())[:1]):
    if df_sum is None: 
        df_sum = pd.read_csv(df_path, encoding='utf-8', low_memory=False)
    else: 
        df_sum.append(pd.read_csv(df_path, encoding='utf-8', low_memory=False))

100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


In [4]:
COLUMNS_TO_USE = [
    'imei', 'name', 'lat', 'lng', 'speed', 'orientation', 'gpstime'
]

COLUMNS_TO_UNIQUE =  [
    'imei', 'name', 'lat', 'lng', 'gpstime'
]

In [9]:
exp_m_df = df_sum[COLUMNS_TO_USE][10_000:10_010]
sub_s_df = df_sum[COLUMNS_TO_USE][10_008:10_012]
# exp_m_df, sub_s_df

In [14]:
df_to_write = exp_m_df.merge(sub_s_df, on=COLUMNS_TO_UNIQUE, how='left', indicator=True)
df_to_write = exp_m_df[(df_to_write['_merge'] == 'left_only').tolist()]
df_to_write

,imei,name,lat,lng,speed,orientation,gpstime
10000,355227045600178,A121,48.305227,25.923100,10.3,144.12,2022-10-01 06:04:50
10001,355227046451688,H89,48.252168,25.934432,37.3,274.27,2022-10-01 06:04:50
10002,355228042084283,A207,48.291015,25.941222,23.4,101.37,2022-10-01 06:04:51
10003,355227046450896,H18,48.309467,25.918255,4.8,170.29,2022-10-01 06:04:52
10004,355227046451407,H68,48.274440,25.945055,10.2,40.77,2022-10-01 06:04:52
10005,355227045526902,A85,48.274265,25.879652,16.7,112.44,2022-10-01 06:04:53
10006,355227044854750,A22,48.324613,25.934222,0.0,0.00,2022-10-01 06:04:53
10007,355227046451019,H30,48.292463,25.936103,0.0,301.05,2022-10-01 06:04:53


In [13]:
(df_to_write['_merge'] == 'left_only').tolist()

[True, True, True, True, True, True, True, True, False, False]

In [4]:
file_path_list = list(Path("D:/projects/pet_project/tables").iterdir())
file_path_list = sorted(file_path_list,
                        key=lambda p: datetime.strptime(p.name[9:-4], '%d_%b_%Y'))

In [5]:
file_path_list[-8:]

[WindowsPath('D:/projects/pet_project/tables/data_for_01_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_02_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_03_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_04_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_05_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_06_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_07_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_08_JAN_2023.csv')]

In [4]:
# df_sum[df_sum.duplicated()]

In [6]:
from datetime import datetime, timedelta
import pytz

import json
import requests
import time

from PTETA.utils.transport.TransportOperator import TransportOperator
from PTETA.utils.transport.TransportRoute import TransportRoute
from PTETA.utils.transport.TransportVehicle import TransportVehicle
from PTETA.utils.transport.TransportAVLData import TransportAVLData
from psycopg2.extensions import connection as Connection
import psycopg2

from apscheduler.schedulers.background import BackgroundScheduler
from PTETA.utils.transport.BaseDBAccessDataclass import BaseDBAccessDataclass
from typing import List, Union
from PTETA.utils.transport.BaseDBAccessDataclass import BaseDBAccessDataclass

In [7]:
response_prev = dict()

try: 
    del TransGPSCVMonitor
except: 
    pass

class TransGPSCVMonitor:
    db_connection: Connection = None

    operator_to_id: dict = dict()
    route_to_id: dict = dict()
    vehicle_to_id: dict = dict()

    objects_unique = {
        TransportOperator: set(),
        TransportRoute: set(),
        TransportVehicle: set()
    }

    def __init__(self, connection_config: dict, **kwarg: dict) -> None:
        self.db_connection = psycopg2.connect(**connection_config)

        self.reload_operators()
        self.reload_routes()
        self.reload_vehicles()

        self.datetime_format = '%Y-%m-%d %H:%M:%S'

        self.REQUEST_URI = \
            kwarg.get('REQUEST_URI', 'http://www.trans-gps.cv.ua/map/tracker/?selectedRoutesStr=')
        self.START_DATE = \
            kwarg.get('START_DATE', (datetime.now() - timedelta(days=1)).strftime(self.datetime_format))
        self.END_DATE = \
            kwarg.get('END_DATE', (datetime.now() + timedelta(days=30)).strftime(self.datetime_format))
        self.REQ_TIME_DELTA = kwarg.get('REQ_TIME_DELTA', 1.1)

    def reload_operators(self):
        operator_list = TransportOperator.get_table(self.db_connection)
        self.objects_unique[TransportOperator] = set(operator_list)
        self.operator_to_id = dict({operator: operator.id for operator in operator_list})

    def reload_routes(self):
        route_list = TransportRoute.get_table(self.db_connection)
        self.objects_unique[TransportRoute] = set(route_list)
        self.route_to_id = dict({route: route.id for route in route_list})

    def reload_vehicles(self):
        vehicle_list = TransportVehicle.get_table(self.db_connection)
        self.objects_unique[TransportVehicle] = set(vehicle_list)
        self.vehicle_to_id = dict({vehicle: vehicle.id for vehicle in vehicle_list})

    @classmethod
    def request_data(cls, request_uri='http://www.trans-gps.cv.ua/map/tracker/?selectedRoutesStr='):
        dt_now = datetime.now()
        dt_tz_now = datetime.utcnow().replace(tzinfo=pytz.utc)

        try:
            request = requests.get(request_uri)
            if (request is None) or (request.text is None):
                return
            response_cur = json.loads(request.text)

            global response_prev

            keys_prev = set(response_prev.keys())
            keys_cur = set(response_cur.keys())

            optimized_data_list = list()
            for imei in keys_prev.intersection(keys_cur):
                if response_prev[imei]['gpstime'] != response_cur[imei]['gpstime']:
                    response_cur[imei]['response_datetime'] = dt_tz_now
                    optimized_data_list += [response_cur[imei]]

            for imei in keys_cur.difference(keys_prev):
                response_cur[imei]['response_datetime'] = dt_tz_now
                optimized_data_list += [response_cur[imei]]

            response_prev = response_cur
        except (requests.Timeout, requests.ConnectionError, requests.HTTPError) as err:
            print(f"{dt_now.strftime('%Y-%m-%d %H;%M;%S')} : error while trying to GET data\n"
                  f"\t{err}\n")

        #         print(dt_now.strftime('%Y-%m-%d %H;%M;%S'), len(optimized_data_list))
        return optimized_data_list

    def get_new_objs(self, obj_list: List[BaseDBAccessDataclass]) -> List[BaseDBAccessDataclass]:
        seen_obj = set()
        seen_add = seen_obj.add
        unique_obj_list = [x for x in obj_list if not (x in seen_obj or seen_add(x))]

        return [obj for obj in unique_obj_list
                if obj not in self.objects_unique[obj.__class__]]

    def update_db(self, obj_list: List[BaseDBAccessDataclass]):
        current_class = obj_list[0].__class__
        are_in_db_list = current_class.are_in_table(self.db_connection, obj_list)

        obj_to_insert = [obj
                         for obj, is_in in zip(obj_list, are_in_db_list)
                         if not is_in]
        
        for obj in obj_to_insert: 
            obj.insert_many_in_table(self.db_connection, [obj])
            if isinstance(obj, TransportOperator):
                self.reload_operators()
            elif isinstance(obj, TransportRoute):
                self.reload_routes()
            elif isinstance(obj, TransportVehicle):
                self.reload_vehicles()

    @classmethod
    def row_validation(cls, row: dict) -> bool:
        if (row['lat'] is None) or (row['lng'] is None):
            return False
        return True
                
    @classmethod
    def decompose_response(cls, response: List[dict], valid_fn=None) -> Union:
        if valid_fn is None:
            valid_fn=cls.row_validation
            
        operator_list, route_list = list(), list()
        vehicle_list, avl_data_list = list(), list()

        for row in response:
            if not valid_fn(row): 
                continue
            operator_list.append(TransportOperator.from_response_row(row))
            route_list.append(TransportRoute.from_response_row(row))
            vehicle_list.append(TransportVehicle.from_response_row(row))
            avl_data_list.append(TransportAVLData.from_response_row(row))

        return operator_list, route_list, vehicle_list, avl_data_list

    def write_to_db(self, response):
        operator_list, route_list, vehicle_list, avl_data_list = self.decompose_response(response)

        for obj_list in [operator_list, route_list, vehicle_list]:
            new_obj = self.get_new_objs(obj_list)
            if new_obj:
                print(f"There are {len(new_obj)} new {new_obj[0].__class__} to inserted in DB")
                self.update_db(new_obj)

        for i, (vehicle, route) in enumerate(zip(vehicle_list, route_list)):
            avl_data_list[i].vehicle_id = self.vehicle_to_id[vehicle]
            avl_data_list[i].route_id = self.route_to_id[route]

        TransportAVLData.insert_many_in_table(self.db_connection, avl_data_list)

    def run(self):
        scheduler = BackgroundScheduler(job_defaults={'max_instances': 8})
        scheduler.add_job(
            self.request_data,
            'interval',
            seconds=self.REQ_TIME_DELTA,
            end_date=self.END_DATE,
            id='listener')

        scheduler.start()

        try:
            print('Scheduler started!')
            while 1:
                time.sleep(10)
                print(datetime.now())
        except KeyboardInterrupt:
            if scheduler.state:
                scheduler.shutdown()

In [8]:
connection_config= dict({
    'host': os.environ['RDS_HOSTNAME'],
    'database': "pteta_db",
    'user': "postgres",
    'password': os.environ['RDS_PTETA_DB_PASSWORD']
})

monitor = TransGPSCVMonitor(connection_config)

req = monitor.request_data()
len(req)

56

In [9]:
file_path_list[-8 + 2 + 3:]

[WindowsPath('D:/projects/pet_project/tables/data_for_06_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_07_JAN_2023.csv'),
 WindowsPath('D:/projects/pet_project/tables/data_for_08_JAN_2023.csv')]

In [10]:
BATCH_SIZE = 1_000
# BATCH_SIZE = 10
connection_config= dict({
    'host': os.environ['RDS_HOSTNAME'],
    'database': "pteta_db",
    'user': "postgres",
    'password': os.environ['RDS_PTETA_DB_PASSWORD']
})

monitor = TransGPSCVMonitor(connection_config)

# file_path_list =list(Path("../data/local/tables/").iterdir())
file_path_list =list(Path("D:/projects/pet_project/tables").iterdir())
file_path_list = sorted(file_path_list, 
                        key=lambda p: datetime.strptime(p.name[9:-4], '%d_%b_%Y') )

for df_path in tqdm(file_path_list[-8 + 2 + 4:]):
    print(df_path.name)
    
    df_sum = pd.read_csv(df_path, encoding='utf-8', low_memory=False)
        
    if "response_datetime" not in df_sum: 
        df_sum['response_datetime'] = None

    if not df_sum['imei'].dtype == 'O':
        df_sum['imei'] = df_sum['imei'].astype(str, copy=True)

    if not df_sum['busNumber'].dtype == 'O':
        df_sum['busNumber'] = df_sum['busNumber'].astype(str, copy=True)
        
    df_cur = df_sum[:]

    batch_tqdm = tqdm(df_cur.groupby(np.arange(len(df_cur)) // BATCH_SIZE), 
                     miniters=40)
    for batch_number, batch_df in batch_tqdm:
        batch_df = batch_df.where(pd.notnull(batch_df), None)
        batch_df = batch_df[batch_df['lat'].notna() | batch_df['lng'].notna()]
        monitor.write_to_db(batch_df.to_dict('records'))

  0%|          | 0/2 [00:00<?, ?it/s]

data_for_07_JAN_2023.csv



 50%|█████     | 1/2 [02:12<02:12, 132.38s/it]

data_for_08_JAN_2023.csv



100%|██████████| 2/2 [04:11<00:00, 125.80s/it]


In [28]:

clmns = list(df_sum.columns)
clmns.remove('gpstime')
# df_sum[df_sum['imei'] == 355227045540176][clmns]
df_sum.drop_duplicates(clmns)

# cols = ["routeId", 'routeName', 'routeColour']
# route_list = [TransportRoute.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]

,id,imei,name,stateCode,stateName,lat,lng,speed,orientation,gpstime,routeId,routeName,routeColour,inDepo,busNumber,perevId,perevName,remark,online,idBusTypes
0,391,355227046453387,H75,used,used,48.325683,25.896533,0.0,122.05,2022-10-01 23:00:57,20,A,navy,False,3627,1,Денисівка,3627 DNSNTNK,True,1
1,386,355227046451407,H68,used,used,48.265903,25.989537,0.0,0.00,2022-10-01 23:02:19,42,39,coral,True,0855,13,Олег Валентинович 39 марш,0855 DNSNTNK,True,1
2,342,355227046451027,H24,used,used,48.324682,25.933822,0.0,0.00,2022-10-01 23:02:23,3,12,blue,True,0738,1,Денисівка,0738 DNSNTNK,True,1
3,294,355228042084283,A207,used,used,48.249142,25.918088,0.0,0.00,2022-10-01 23:03:14,42,39,coral,False,1032,13,Олег Валентинович 39 марш,1032 DNSNK,True,1
4,165,355227045540176,A83,used,used,48.320132,25.940298,0.0,0.00,2022-10-01 23:03:26,41,10A,black,False,5150,7,Віталій / Олег Бойко,5150 DNS,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173374,243,355227045600988,A158,used,used,48.251908,25.938167,11.8,101.45,2022-10-02 22:39:14,33,8T,coral,False,324,6,ЧТУ,Тролейбус 324 DNSNK,True,2
173375,243,355227045600988,A158,used,used,48.251842,25.938617,10.0,98.09,2022-10-02 22:39:24,33,8T,coral,False,324,6,ЧТУ,Тролейбус 324 DNSNK,True,2
173376,243,355227045600988,A158,used,used,48.251825,25.939043,11.6,90.99,2022-10-02 22:39:34,33,8T,coral,False,324,6,ЧТУ,Тролейбус 324 DNSNK,True,2
173378,243,355227045600988,A158,used,used,48.251743,25.939465,10.8,127.16,2022-10-02 22:39:44,33,8T,coral,False,324,6,ЧТУ,Тролейбус 324 DNSNK,True,2


In [45]:
# monitor.vehicle_to_id
# TransportVehicle(id=154, imei='355227046451266', name='H53', bus_number='2011', remark='Тролейбус 2011 DNSNTNK', perev_id=6): 228,
# TransportVehicle(id=Non, imei='355227046451266', name='H53', bus_number=2011, remark='Тролейбус 2011 DNSNTNK', perev_id=6)

{TransportVehicle(id=100, imei='355227045600830', name='A178', bus_number='310', remark='Тролейбус 310 DNSNK', perev_id=6): 230,
 TransportVehicle(id=101, imei='355227046451662', name='H76', bus_number='350', remark='Тролейбус 350 DNSNTNK', perev_id=6): 101,
 TransportVehicle(id=102, imei='355227045369527', name='A6', bus_number='3557', remark='3557 DNSNK ', perev_id=12): 232,
 TransportVehicle(id=103, imei='355227045540176', name='A83', bus_number='5150', remark=' 5150 DNS', perev_id=7): 229,
 TransportVehicle(id=104, imei='355227046453387', name='H75', bus_number='3627', remark='3627 DNSNTNK', perev_id=1): 227,
 TransportVehicle(id=105, imei='355228042084283', name='A207', bus_number='1032', remark='1032 DNSNK', perev_id=13): 236,
 TransportVehicle(id=106, imei='355227045371655', name='A1', bus_number='6513', remark='6513 DNSNK', perev_id=7): 106,
 TransportVehicle(id=107, imei='355227046451407', name='H68', bus_number='0855', remark='0855 DNSNTNK', perev_id=13): 107,
 TransportVehic

In [52]:
monitor.objects_unique[TransportVehicle]

{TransportVehicle(id=100, imei='355227045600830', name='A178', bus_number='310', remark='Тролейбус 310 DNSNK', perev_id=6),
 TransportVehicle(id=101, imei='355227046451662', name='H76', bus_number='350', remark='Тролейбус 350 DNSNTNK', perev_id=6),
 TransportVehicle(id=102, imei='355227045369527', name='A6', bus_number='3557', remark='3557 DNSNK ', perev_id=12),
 TransportVehicle(id=103, imei='355227045540176', name='A83', bus_number='5150', remark=' 5150 DNS', perev_id=7),
 TransportVehicle(id=104, imei='355227046453387', name='H75', bus_number='3627', remark='3627 DNSNTNK', perev_id=1),
 TransportVehicle(id=105, imei='355228042084283', name='A207', bus_number='1032', remark='1032 DNSNK', perev_id=13),
 TransportVehicle(id=106, imei='355227045371655', name='A1', bus_number='6513', remark='6513 DNSNK', perev_id=7),
 TransportVehicle(id=107, imei='355227046451407', name='H68', bus_number='0855', remark='0855 DNSNTNK', perev_id=13),
 TransportVehicle(id=108, imei='355227046578332', name=

'f'

## TransportRoute

In [7]:
TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]

['route_name', 'route_colour']

In [28]:
cols = ["routeId", 'routeName', 'routeColour']
route_list = [TransportRoute.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(route_list), route_list[:3]
TransportRoute.insert_many_in_table(conn, route_list)

In [32]:
TransportRoute.get_table(conn)

[TransportRoute(id=37, name='T', colour='coral'),
 TransportRoute(id=31, name='6/6a', colour='deeppink'),
 TransportRoute(id=21, name='38', colour='deeppink'),
 TransportRoute(id=41, name='10A', colour='black'),
 TransportRoute(id=20, name='A', colour='navy'),
 TransportRoute(id=42, name='39', colour='coral'),
 TransportRoute(id=23, name='19', colour='teal'),
 TransportRoute(id=19, name='10', colour='black'),
 TransportRoute(id=11, name='9', colour='magenta'),
 TransportRoute(id=2, name='11', colour='green'),
 TransportRoute(id=6, name='4', colour='magenta'),
 TransportRoute(id=27, name='3/3a', colour='green'),
 TransportRoute(id=16, name='2', colour='green'),
 TransportRoute(id=4, name='5', colour='orange'),
 TransportRoute(id=3, name='12', colour='blue'),
 TransportRoute(id=12, name='20', colour='maroon'),
 TransportRoute(id=7, name='6', colour='sienna'),
 TransportRoute(id=10, name='34', colour='navy'),
 TransportRoute(id=45, name='1', colour='navy'),
 TransportRoute(id=9, name='27'

In [13]:
# df_sum[TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]].value_counts()

In [27]:
conn.rollback()

In [45]:
SQL_big_req = " ".join([create_sql_req(t) for t in trans_vehicle_list])

In [29]:
route_list = TransportRoute.get_table(conn)[:]
len(route_list)

31

In [30]:
for r in route_list[::2]: 
    r.id += 1
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [31]:
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [14]:
for r in route_list: 
    r.name += "_"
    r.id += 100

In [16]:
# sql = f"""INSERT INTO pteta.route("id", "routeName", "routeColour") VALUES """ + \
#               ", ".join([f"""({obj.id}, '{obj.name}', '{obj.colour}')"""
#                         for obj in route_list]) + ";"
# sql

In [17]:
# route_list

In [18]:
TransportRoute.insert_many_in_table(conn, route_list)

## Test TransportVehicle

In [33]:
TransportVehicle.__insert_columns__()

'"imei", "name", "bus_number", "remark", "perev_id"'

In [45]:
cols = ["imei", "name", "busNumber", "remark", "perevId"]
vehicle_list = [TransportVehicle.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(vehicle_list), vehicle_list[:3]
TransportVehicle.insert_many_in_table(conn, vehicle_list)

In [47]:
vehicle_list = TransportVehicle.get_table(conn)
len(vehicle_list)

99

In [48]:
TransportVehicle.are_in_table(conn, vehicle_list[:5])

[True, True, True, True, True]

In [49]:
vehicle_list[5].is_in_table(conn)

True

In [50]:
obj = vehicle_list[0]
obj.imei += "-"
obj.is_in_table(conn)

False

In [51]:
obj.insert_in_table(conn)

In [52]:
for v in vehicle_list[:10]: 
    v.imei += '_'

In [15]:
TransportVehicle.insert_many_in_table(conn, vehicle_list[:10])

In [31]:
obj.insert_in_table(conn)
obj.is_in_table(conn)

In [14]:
vehicle_list[0].id = None
print(vehicle_list[0])
vehicle_list[0].update_id_from_table(conn)
print(vehicle_list[0])

TransportVehicle(id=None, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)
TransportVehicle(id=1, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)


In [34]:
for v in vehicle_list[:10]: 
    v.imei += '__'

vehicle_list[:13]

[TransportVehicle(id=222, imei='355227045600830-__', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37),
 TransportVehicle(id=2, imei='355227046451662__', name='H76', busNumber='350', remark='Тролейбус 350 DNSNTNK', perevId=6, routeId=31),
 TransportVehicle(id=3, imei='355227045369527__', name='A6', busNumber='3557', remark='3557 DNSNK ', perevId=12, routeId=21),
 TransportVehicle(id=4, imei='355227045540176__', name='A83', busNumber='5150', remark=' 5150 DNS', perevId=7, routeId=41),
 TransportVehicle(id=5, imei='355227046453387__', name='H75', busNumber='3627', remark='3627 DNSNTNK', perevId=1, routeId=20),
 TransportVehicle(id=6, imei='355228042084283__', name='A207', busNumber='1032', remark='1032 DNSNK', perevId=13, routeId=42),
 TransportVehicle(id=7, imei='355227045371655__', name='A1', busNumber='6513', remark='6513 DNSNK', perevId=7, routeId=23),
 TransportVehicle(id=8, imei='355227046451407__', name='H68', busNumber='0855', remark='0855 DNSNTNK

## TransportOperator

In [41]:
operator_list = TransportOperator.get_table(conn)
len(operator_list)

6

In [37]:
TransportOperator.__insert_columns__()

'id, "perev_name"'

In [40]:
cols = ['perevId', "perevName"]
operator_list = [TransportOperator.from_response_row(row) 
              for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(operator_list), operator_list[:3]
TransportOperator.insert_many_in_table(conn, operator_list)

In [42]:
TransportOperator.are_in_table(conn, operator_list[:])

[True, True, True, True, True, True]

In [43]:
operator_list[0].is_in_table(conn)

True

In [44]:
obj = operator_list[0]
obj.id += 100
obj.name = "-_-" + obj.name
obj.is_in_table(conn)

False

In [14]:
obj.insert_in_table(conn)

In [15]:
for op in operator_list: 
    op.id += 100
    op.name = "-_-" + op.name
    
TransportOperator.insert_many_in_table(conn, operator_list)

## TransportAVLData

In [9]:
avl_data_list = TransportAVLData.get_table(conn)

In [11]:
# avl_data_list

In [14]:
%%timeit
TransportAVLData.are_in_table(conn, avl_data_list)

57.4 ms ± 3.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
avl_data_list[0].is_in_table(conn)

42.1 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
